In [ ]:
import autograd.numpy as np
from autograd import grad
import autograd.numpy.random as npr
from autograd.numpy.linalg import solve
import autograd.scipy.stats.multivariate_normal as mvn
from autograd import value_and_grad
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def make_gp_funs(cov_func, num_cov_params):
    """Functions that perform Gaussian process regression.
       cov_func has signature (cov_params, x, x')"""
    
    def unpack_kernel_params(params):
        mean        = params[0]
        cov_params  = params[2:]
        noise_scale = np.exp(params[1]) + 0.0001
        return mean, cov_params, noise_scale
    
    def predict(params, x, y, xstar):
        """Returns the predictive mean and covariance at locations xstar,
           of the latent function value f (without observation noise)."""
        mean, cov_params, noise_scale = unpack_kernel_params(params)
        # Calculate covariances
        # Calculate predictive mean
        # Calculate predictive covariance
        return pred_mean, pred_cov
    
    def log_marginal_likelihood(params, x, y):
        mean, cov_params, noise_scale = unpack_kernel_params(params)
        # Calculate *observation* covariance
        # Enumerate 'prior' mean
        return mvn.logpdf(y, prior_mean, cov_y_y)
    
    return num_cov_params + 2, predict, log_marginal_likelihood

In [ ]:
# Define a covariance function.
def rbf_covariance(kernel_params, x, xp):
    # params of: output scale, lengthscale (need to constrain to be positive!)
    return 

In [ ]:
def build_toy_dataset(D=1, n_data=50, noise_std=0.2):
    rs = npr.RandomState(0)
    inputs  = np.concatenate([np.linspace(0, 4, num=n_data/2),
                              np.linspace(6, 8, num=n_data/2)])
    targets = (np.cos(inputs/4) + np.sin(inputs) + rs.randn(n_data) * noise_std) / 2.0
    inputs = (inputs - 4.0) / 2.0
    inputs  = inputs.reshape((len(inputs), D))
    return inputs, targets

D = 1
n_data = 50
noise_std=0.2
X, y = build_toy_dataset(D=D, n_data=n_data, noise_std=noise_std)

plt.plot(X,y,'.')

In [ ]:
## SPOILER ALERT ##
# Code below is super useful, but does all the rest of the work for you!!!

In [ ]:
##Use this nifty code to optimize
# Build model and objective function.
num_params, predict, log_marginal_likelihood = make_gp_funs(rbf_covariance, num_cov_params=D + 1)
objective = lambda params: -log_marginal_likelihood(params, X, y)
# Initialize covariance parameters
rs = npr.RandomState(0)
init_params = 0.1 * rs.randn(num_params)
# Optimize using conjugate gradients
opt_params = minimize(value_and_grad(objective), init_params, jac=True, method='CG')
params = opt_params.x

In [ ]:
## Use this nifty code to plot
# Show posterior marginals.
plot_xs = np.reshape(np.linspace(-7, 7, 300), (300,1))
pred_mean, pred_cov = predict(params, X, y, plot_xs)
marg_std = np.sqrt(np.diag(pred_cov))
plt.plot(plot_xs, pred_mean, 'b')
plt.fill(np.concatenate([plot_xs, plot_xs[::-1]]),
        np.concatenate([pred_mean - 1.96 * marg_std,
                       (pred_mean + 1.96 * marg_std)[::-1]]),
        alpha=.15, fc='Blue', ec='None')
plt.plot(X,y,'.')

# Show samples from posterior.
rs = npr.RandomState(0)
sampled_funcs = rs.multivariate_normal(pred_mean, pred_cov, size=10)
plot.plot(plot_xs, sampled_funcs.T)

plt.plot(X, y, 'kx')
plt.set_ylim([-1.5, 1.5])
plt.set_xticks([])
plt.set_yticks([])